# Data analysis of the models results

This notebook aims to provide analytical tools to work on the damage functions modeled in the WILIAM model folder. 

In [1]:
import pysd
import pandas as pd

In [9]:
reload = True

if reload:
    model = pysd.read_vensim(r'C:\\Users\\gabriel.genelot\\Documents\\damage-functions\\WILIAM_v1.3\\WILIAM.mdl')
else:
    pass

VisitationError: SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<string>, line 1)

Parse tree:
<Node called "string" matching "'C:\Users\gabriel.genelot\Documents\damage-functions\WILIAM_v1.3\df_parameters\FUND\output\wiliam_table_a.csv'">  <-- *** We were here. ***
    <Node matching "'">
    <Node matching "C:\Users\gabriel.genelot\Documents\damage-functions\WILIAM_v1.3\df_parameters\FUND\output\wiliam_table_a.csv">
        <RegexNode matching "C">
        <RegexNode matching ":">
        <RegexNode matching "\">
        <RegexNode matching "U">
        <RegexNode matching "s">
        <RegexNode matching "e">
        <RegexNode matching "r">
        <RegexNode matching "s">
        <RegexNode matching "\">
        <RegexNode matching "g">
        <RegexNode matching "a">
        <RegexNode matching "b">
        <RegexNode matching "r">
        <RegexNode matching "i">
        <RegexNode matching "e">
        <Regex